## Logistic Regression to predict absenteeism

### Libraries

In [40]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from sklearn.base import BaseEstimator, TransformerMixin

import pickle

In [2]:
# Load the preprocessed CSV data
data_preprocessed = pd.read_csv('Absenteeism_preprocessed.csv')

In [3]:
data_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


### Create the targets

In [4]:
# Find the median of 'Absenteeism Time in Hours'
data_preprocessed['Absenteeism Time in Hours'].median()

3.0

In [5]:
# Create targets for logistic regression

# Step 1: find a way to say if someone is 'being absent too much' or not
    # Solution: take the median of the dataset as a cut-off line
    # The dataset will be balanced (there will be roughly equal number of 0s and 1s for the logistic regression)

# Add Targets
# Moderately absent <= 3 -- Target -- > 0
# Excessively absent >= 4 -- Target --> 1

# If value in Absenteeism Time in Hours is grater than median (in this case 3), put 1, else put 0.
targets = np.where(data_preprocessed['Absenteeism Time in Hours'] > 
                   data_preprocessed['Absenteeism Time in Hours'].median(), 1, 0)

In [6]:
# Add new column Excessive Absenteeism
data_preprocessed['Excessive Absenteeism'] = targets

In [7]:
data_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Excessive Absenteeism
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4,1
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2,0


### A comment on the targets

In [8]:
# Check if dataset is balanced (what % of targets are 1s)
# targets.sum() will give us the number of 1s that there are
# the shape[0] will give us the length of the targets array
targets.sum() / targets.shape[0]

0.45571428571428574

In [9]:
# Note: A 60-40 split will usually work equally well for a logistic regression

In [10]:
# Checkpoint
data_with_targets = data_preprocessed.drop(['Absenteeism Time in Hours'], axis=1)

### Select the inputs for the regression

In [11]:
data_with_targets.shape

(700, 15)

In [12]:
# Selects all rows and all columns but the last one
data_with_targets.iloc[:,:-1]

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,5,2,179,22,40,237.656,22,1,2,0
696,1,0,0,0,5,2,225,26,28,237.656,24,0,1,2
697,1,0,0,0,5,3,330,16,28,237.656,25,1,0,0
698,0,0,0,1,5,3,235,16,32,237.656,25,1,0,0


In [13]:
# Create a variable that will contain the inputs
unscaled_inputs = data_with_targets.iloc[:,:-1]

### Standardize the data

In [14]:
# Create empty StandardScaler object. 'absenteeism_scaler' will be used to substract the mean and divide
# by the standard deviation variablewise (featurewise)

# Hide for CustomScaler purpose
absenteeism_scaler = StandardScaler()

In [15]:
# Calculate and store the mean and the standard deviation
absenteeism_scaler.fit(unscaled_inputs)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [16]:
# .transform() does the actual scaling
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)

### Split the data into train & test and shuffle

In [17]:
# Split the data into inputs and targets.
# Output: array 1: a training dataset with inputs
#         array 2: a training dataset with targets
#         array 3: a test dataset with inputs
#         array 4: a test dataset with targets
train_test_split(scaled_inputs, targets)

[array([[-0.57735027, -0.09298136, -0.31448545, ..., -0.44798003,
          0.88046927,  2.8430157 ],
        [ 1.73205081, -0.09298136, -0.31448545, ..., -0.44798003,
          0.88046927,  0.26848661],
        [-0.57735027, -0.09298136,  3.17979734, ..., -0.44798003,
         -0.01928035,  1.12666297],
        ...,
        [ 1.73205081, -0.09298136, -0.31448545, ...,  2.23224237,
          0.88046927,  1.12666297],
        [-0.57735027, -0.09298136, -0.31448545, ..., -0.44798003,
         -0.01928035,  1.12666297],
        [-0.57735027, -0.09298136, -0.31448545, ..., -0.44798003,
          0.88046927,  0.26848661]]),
 array([[ 1.73205081, -0.09298136, -0.31448545, ...,  2.23224237,
         -0.91902997, -0.58968976],
        [-0.57735027, -0.09298136, -0.31448545, ..., -0.44798003,
         -0.01928035, -0.58968976],
        [-0.57735027, -0.09298136, -0.31448545, ..., -0.44798003,
         -0.91902997, -0.58968976],
        ...,
        [-0.57735027, -0.09298136, -0.31448545, ...,  

In [18]:
# Random_state is the method that always shuffle the observations in the same 'random' way
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, train_size = 0.8, random_state = 20)

In [19]:
print(x_train.shape, y_train.shape)

(560, 14) (560,)


In [20]:
print(x_test.shape, y_test.shape)

(140, 14) (140,)


### Logistic regression

In [21]:
# Training the model
reg = LogisticRegression()

In [22]:
reg.fit(x_train, y_train)

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
# Based on the used data, the model learned to classify ~80% (0,7804) of the observations correctly
reg.score(x_train, y_train)

0.7839285714285714

### Manually check the accuracy

In [24]:
# Predict class labels (logistic regression outputs) for given input sample
model_outputs = reg.predict(x_train)

In [25]:
np.sum(model_outputs == y_train)

439

In [26]:
# Accuracy = correct predicitions / # observations
np.sum(model_outputs == y_train) / model_outputs.shape[0]

0.7839285714285714

### Finding the intercept and coefficients

In [27]:
# Intercept
reg.intercept_

array([-0.21957576])

In [28]:
# Coefficient
reg.coef_

array([[ 2.07452213,  0.33466331,  1.56097327,  1.32750188,  0.1878226 ,
        -0.07060017,  0.70649077, -0.03995965, -0.2009517 , -0.00462411,
         0.31937669, -0.13558227,  0.38168134, -0.33338757]])

In [29]:
feature_name = unscaled_inputs.columns.values

In [30]:
summary_table = pd.DataFrame(columns = ['Feature name'], data = feature_name)

# Use np.transpose because np.array is a row 
summary_table['Coefficient'] = np.transpose(reg.coef_)

summary_table

,Feature name,Coefficient
0,Reason_1,2.074522
1,Reason_2,0.334663
2,Reason_3,1.560973
3,Reason_4,1.327502
4,Month Value,0.187823
5,Day of the Week,-0.070600
6,Transportation Expense,0.706491
7,Distance to Work,-0.039960
8,Age,-0.200952
9,Daily Work Load Average,-0.004624


In [31]:
# Add Intercept as a first row of table

# Number Features with the same numer as in the basic table
summary_table.index = summary_table.index + 1
# Add new row for Intercept
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
# Sort index
summary_table = summary_table.sort_index()

summary_table

,Feature name,Coefficient
0,Intercept,-0.219576
1,Reason_1,2.074522
2,Reason_2,0.334663
3,Reason_3,1.560973
4,Reason_4,1.327502
5,Month Value,0.187823
6,Day of the Week,-0.070600
7,Transportation Expense,0.706491
8,Distance to Work,-0.039960
9,Age,-0.200952


### Interpreting the coefficients

In [32]:
# Standardized coefficients: the coefficients of a regression where all variables have been standardized

# Beta 0 = bias/intercept

# Beta 1,...,Beta n = Coefficient/weight
# The closer the weights to 0, the smaller the weight
# Whichever weights is bigger, its corresponding feature is more important

In [33]:
summary_table['Odds_ratio'] = np.exp(summary_table.Coefficient)
summary_table

,Feature name,Coefficient,Odds_ratio
0,Intercept,-0.219576,0.802859
1,Reason_1,2.074522,7.960741
2,Reason_2,0.334663,1.397470
3,Reason_3,1.560973,4.763455
4,Reason_4,1.327502,3.771610
5,Month Value,0.187823,1.206619
6,Day of the Week,-0.070600,0.931834
7,Transportation Expense,0.706491,2.026866
8,Distance to Work,-0.039960,0.960828
9,Age,-0.200952,0.817952


In [34]:
summary_table.sort_values('Odds_ratio', ascending=False)

# A feature is not particularly important:
    # if its coefficient is around 0
    # if its odds ratio is around 1

# A weight (coefficient) of 0 implies that no matter the feature value, we will multiply it by 0 (in the model)
# For a unit change in the standardized feature, the odds increase by a multiple equal to the odds ratio (1 = no change)

,Feature name,Coefficient,Odds_ratio
1,Reason_1,2.074522,7.960741
3,Reason_3,1.560973,4.763455
4,Reason_4,1.327502,3.771610
7,Transportation Expense,0.706491,2.026866
13,Children,0.381681,1.464745
2,Reason_2,0.334663,1.397470
11,Body Mass Index,0.319377,1.376270
5,Month Value,0.187823,1.206619
10,Daily Work Load Average,-0.004624,0.995387
8,Distance to Work,-0.039960,0.960828


### Testing the model

In [35]:
reg.score(x_test, y_test)

0.7357142857142858

In [37]:
# Based on data that the model has never seen before, in 73,6% of the cases, 
# the model will predict (correctly) if the person is going to be excessively absent

# * often the test accuracy is 10-20% lower than the train accuracy (due to overfitting)

In [38]:
# Instead of 0 and 1, get the PROBABILITY of an output bein 0 or 1
predicted_proba = reg.predict_proba(x_test)

In [39]:
# The first column = probability of being 0
# The second column = probability of being 1
predicted_proba

array([[0.73825966, 0.26174034],
       [0.58795033, 0.41204967],
       [0.36712696, 0.63287304],
       [0.81162265, 0.18837735],
       [0.06127301, 0.93872699],
       [0.30690384, 0.69309616],
       [0.29493568, 0.70506432],
       [0.08079407, 0.91920593],
       [0.80400642, 0.19599358],
       [0.75999392, 0.24000608],
       [0.4622669 , 0.5377331 ],
       [0.15450706, 0.84549294],
       [0.04604734, 0.95395266],
       [0.72947484, 0.27052516],
       [0.23080142, 0.76919858],
       [0.55067175, 0.44932825],
       [0.52874769, 0.47125231],
       [0.55477826, 0.44522174],
       [0.38143834, 0.61856166],
       [0.0251927 , 0.9748073 ],
       [0.69578142, 0.30421858],
       [0.80421732, 0.19578268],
       [0.3768243 , 0.6231757 ],
       [0.39948746, 0.60051254],
       [0.18176149, 0.81823851],
       [0.75976147, 0.24023853],
       [0.46988997, 0.53011003],
       [0.88978022, 0.11021978],
       [0.13538822, 0.86461178],
       [0.79659382, 0.20340618],
       [0.

### Save the model

In [41]:
with open('log_reg_model', 'wb') as file:
    pickle.dump(reg, file)

In [42]:
with open('scaler','wb') as file:
    pickle.dump(absenteeism_scaler, file)